# 🛍️ Gender-Based Sales Trends in Supermarket Transactions  
### Exploring Consumer Behavior and Ethical Considerations in Business Intelligence

> _How do product preferences and average purchase values differ by gender — and what are the ethical implications of using this data in decision-making?_


### Set up the Notebook

In [80]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output



### Connect to SQLite Database

In [81]:
# Connect to your SQLite DW
conn = sqlite3.connect(r"C:\Users\beths\Documents\Northwest_Missouri_Analytics\BI_Analytics\supermarket_sales\data\dw\supermarket_dw.db")


### Load and Explore Data

In [82]:
# Load tables
df_customers = pd.read_sql("SELECT * FROM customer", conn)
df_products = pd.read_sql("SELECT * FROM product", conn)
df_sales = pd.read_sql("SELECT * FROM sale", conn)

# Merge tables into one analysis DataFrame
df = df_sales.merge(df_customers, on="customer_id").merge(df_products, on="product_id")
df["datetime"] = pd.to_datetime(df["datetime"])
df["month"] = df["datetime"].dt.to_period("M").astype(str)

df.head()


,sale_id,customer_id,product_id,quantity_sold,total,branch,city,datetime,rating,gender,payment_method,product_line,unit_price,tax_pct,month
0,750-67-8428,750-67-8428,750-67-8428,7,548.9715,A,Yangon,2019-01-05 13:08:00,9.1,Female,Ewallet,Health and beauty,74.69,26.1415,2019-01
1,226-31-3081,226-31-3081,226-31-3081,5,80.2200,C,Naypyitaw,2019-03-08 10:29:00,9.6,Female,Cash,Electronic accessories,15.28,3.8200,2019-03
2,631-41-3108,631-41-3108,631-41-3108,7,340.5255,A,Yangon,2019-03-03 13:23:00,7.4,Male,Credit card,Home and lifestyle,46.33,16.2155,2019-03
3,123-19-1176,123-19-1176,123-19-1176,8,489.0480,A,Yangon,2019-01-27 20:33:00,8.4,Male,Ewallet,Health and beauty,58.22,23.2880,2019-01
4,373-73-7910,373-73-7910,373-73-7910,7,634.3785,A,Yangon,2019-02-08 10:37:00,5.3,Male,Ewallet,Sports and travel,86.31,30.2085,2019-02


### Visualize and Analyze Gender-Based Sales Trends

Average Purchase Value by Gender

In [83]:
avg_purchase = df.groupby("gender")["total"].mean().reset_index()
fig = px.bar(avg_purchase, x="gender", y="total", title="Average Purchase Value by Gender")
fig.show()


In [84]:
df.groupby("gender")["total"].mean()

gender
Female    335.095659
Male      310.789226
Name: total, dtype: float64

### Product Preferences by Gender

In [85]:
product_gender = df.groupby(["gender", "product_line"]).size().reset_index(name="count")
fig = px.bar(product_gender, 
             x="product_line", 
             y="count", 
             color="gender", 
             barmode="group", 
             title="Product Preferences by Gender")
fig.update_layout(xaxis_title="Product Line", yaxis_title="Number of Transactions")
fig.show()


### Total Spent per Product Line by Gender

In [86]:
spend_gender = df.groupby(["gender", "product_line"])["total"].sum().reset_index()
fig = px.bar(spend_gender, 
             x="product_line", 
             y="total", 
             color="gender", 
             barmode="group", 
             title="Total Spend by Product Line and Gender")
fig.update_layout(xaxis_title="Product Line", yaxis_title="Total Spend ($)")
fig.show()



In [92]:
# Create the Dash app
app = dash.Dash(__name__)
app.title = "Product Preferences by Gender"

# App layout
app.layout = html.Div([
    html.H1("🛍️ Product Preferences by Gender", style={'backgroundColor': 'white'}),


    html.Div([
        html.Label("Select Gender:"),
        dcc.Dropdown(
            id="gender-dropdown",
            options=[{"label": g, "value": g} for g in df["gender"].unique()],
            value="Female",
            style={'width': '100%'}
        ),
    ], style={'width': '300px', 'margin': 'auto'}),

    dcc.Graph(id="product-bar-chart"),

    html.Div([
        html.P("This dashboard displays product line preferences by gender based on transaction counts."),
        html.P("Use the dropdown above to filter the data by gender."),
    ], style={'textAlign': 'center', 'marginTop': '40px'})
])

# Callback to update bar chart based on selected gender
@app.callback(
    Output("product-bar-chart", "figure"),
    Input("gender-dropdown", "value")
)
def update_chart(selected_gender):
    filtered = df[df["gender"] == selected_gender]
    product_counts = filtered["product_line"].value_counts().reset_index()
    product_counts.columns = ["product_line", "count"]

    fig = px.bar(
        product_counts,
        x="product_line",
        y="count",
        title=f"Product Line Preferences - {selected_gender}",
        labels={"count": "Number of Transactions", "product_line": "Product Line"}
    )

    fig.update_layout(
        template="plotly_white",
        xaxis_title="Product Line",
        yaxis_title="Transactions",
        title_x=0.5,
        font=dict(size=14)
    )
    
    return fig

if __name__ == "__main__":
    app.run(debug=True)




### Average Customer Rating by Gender

In [88]:
rating_gender = df.groupby("gender")["rating"].mean().reset_index()
fig = px.bar(rating_gender, 
             x="gender", 
             y="rating", 
             title="Average Customer Rating by Gender")
fig.update_layout(xaxis_title="Gender", yaxis_title="Average Rating (out of 10)")
fig.show()


### Average Spent per Month by Gender

In [89]:
df["month"] = pd.to_datetime(df["datetime"]).dt.to_period("M").astype(str)
monthly_spend = df.groupby(["month", "gender"])["total"].mean().reset_index()

fig = px.line(monthly_spend, 
              x="month", 
              y="total", 
              color="gender", 
              markers=True,
              title="Monthly Average Purchase Value by Gender")
fig.update_layout(xaxis_title="Month", yaxis_title="Avg Purchase Value ($)")
fig.show()


### 📌 Key Insights

- **Average spend per transaction** is higher for females.
- **Product line preferences** show trends such as females tend to purchase Health and Beauty, while males purchase Electronics.

### ⏳ Timeframe Context & Business Interpretation

This dataset covers a small timeframe of around three months, which is just a **snapshot** of customer behavior.

While the data doesn't support long-term forecasting, it **does provide meaningful insights** for:

- **Quarterly strategy planning** (e.g., Q1 performance review)
- **Short-term campaign effectiveness**
- **Product line experimentation**
- **Customer segmentation tests**

In this snapshot, we can observe:
- Which gender groups are spending more 
- Which product lines are trending among different customer segments
- Whether average transaction value varies across genders

📌 **Insight**: Even in a short timeframe, gender-based behavior shows **consistent patterns** — for example, if a specific product line sees strong preference among a gender group, it may be worth testing targeted promotions in the next quarter.

⚠️ **Caution**: These trends should be validated with longer-term data before making major business decisions, especially those that involve automation or personalization.
